# Expanding Variable Cluster Timeline

Author: Sofia Ingersoll 5 May 2024

## Increaseing Timeline
Loading files in 4, 500 cluster chunks 
`campaign/cgd/tss/projects/PPE/PPEn11_LHC/transient/hist/PPEn11_transient_LHC0113.clm2.h0.1995-02-01-00000.nc`

In [1]:
import xarray as xr
import pickle
from utils import * 

In [2]:
# Request an additional 10 cores of power for processing from the server
client = get_cluster("UCSB0021", cores = 30)
# apply peer2peer network communication across multiple devices
client.cluster

/glade/u/home/singersoll/.conda/envs/meds-py/lib/python3.11/site-packages/distributed/node.py:182: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 40417 instead
  warnings.warn(


Dashboard: https://jupyterhub.hpc.ucar.edu/stable/user/singersoll/proxy/40417/status,Workers: 0
Total threads: 0,Total memory: 0 B
Comm: tcp://128.117.208.67:33915,Workers: 0
Dashboard: https://jupyterhub.hpc.ucar.edu/stable/user/singersoll/proxy/40417/status,Total threads: 0
Started: Just now,Total memory: 0 B


In [3]:
cluster_lists = [
        sorted(glob.glob('/glade/campaign/cgd/tss/projects/PPE/PPEn11_LHC/transient/hist/PPEn11_transient_LHC[0][0-5][0-9][0-9].clm2.h0.1995-02-01-00000.nc')),
        sorted(glob.glob('/glade/campaign/cgd/tss/projects/PPE/PPEn11_LHC/transient/hist/PPEn11_transient_LHC[0][0-5][0-9][0-9].clm2.h0.2000-02-01-00000.nc')),
        sorted(glob.glob('/glade/campaign/cgd/tss/projects/PPE/PPEn11_LHC/transient/hist/PPEn11_transient_LHC[0][0-5][0-9][0-9].clm2.h0.2005-02-01-00000.nc')),
        sorted(glob.glob('/glade/campaign/cgd/tss/projects/PPE/PPEn11_LHC/transient/hist/PPEn11_transient_LHC[0][0-5][0-9][0-9].clm2.h0.2010-02-01-00000.nc'))
    ]

In [4]:
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# ----     cluster reading function       ----
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
#modify the function if you want to pass the parameter
def read_all_simulations(var):
    '''Prepare cluster list and read to create ensemble(group of data)
    Use preprocess to select only certain dimension and a variable'''
    # Define the list of lists
    cluster_lists = [
        sorted(glob.glob('/glade/campaign/cgd/tss/projects/PPE/PPEn11_LHC/transient/hist/PPEn11_transient_LHC[0][0-5][0-9][0-9].clm2.h0.1995-02-01-00000.nc')),
        sorted(glob.glob('/glade/campaign/cgd/tss/projects/PPE/PPEn11_LHC/transient/hist/PPEn11_transient_LHC[0][0-5][0-9][0-9].clm2.h0.2000-02-01-00000.nc')),
        sorted(glob.glob('/glade/campaign/cgd/tss/projects/PPE/PPEn11_LHC/transient/hist/PPEn11_transient_LHC[0][0-5][0-9][0-9].clm2.h0.2005-02-01-00000.nc')),
        sorted(glob.glob('/glade/campaign/cgd/tss/projects/PPE/PPEn11_LHC/transient/hist/PPEn11_transient_LHC[0][0-5][0-9][0-9].clm2.h0.2010-02-01-00000.nc'))
    ]

    all_data = []  # List to store data from all simulations
    
    for cluster_list in cluster_lists:
        # Exclude the first element of the cluster_list
        cluster_list = cluster_list[1:]
        
        # Read each simulation and append to the list
        ds = xr.open_mfdataset(cluster_list,
                               combine='nested',
                               preprocess=lambda ds: ds[['lat', 'lon', 'time', var]],
                               parallel=True)
        all_data.append(ds)
    
    # Combine all the datasets into a single dataset along the 'ens' and 'time' dimensions
    combined_ds = xr.concat(all_data, dim=["ens", "time"])
    
    return combined_ds

In [5]:
read_all_simulations("LNC")

MergeError: conflicting values for variable 'LNC' on objects to be combined. You can skip this check by specifying compat='override'.

### testing out error suggestion, but i think it's bc I'm not calling real files

In [8]:
def read_all_simulations(var):
    '''Prepare cluster list and read to create ensemble(group of data)
    Use preprocess to select only certain dimension and a variable'''
    # Define the list of lists
    cluster_lists = [
        sorted(glob.glob('/glade/campaign/cgd/tss/projects/PPE/PPEn11_LHC/transient/hist/PPEn11_transient_LHC[0][0-5][0-9][0-9].clm2.h0.1995-02-01-00000.nc')),
        sorted(glob.glob('/glade/campaign/cgd/tss/projects/PPE/PPEn11_LHC/transient/hist/PPEn11_transient_LHC[0][0-5][0-9][0-9].clm2.h0.2000-02-01-00000.nc')),
        sorted(glob.glob('/glade/campaign/cgd/tss/projects/PPE/PPEn11_LHC/transient/hist/PPEn11_transient_LHC[0][0-5][0-9][0-9].clm2.h0.2005-02-01-00000.nc')),
        sorted(glob.glob('/glade/campaign/cgd/tss/projects/PPE/PPEn11_LHC/transient/hist/PPEn11_transient_LHC[0][0-5][0-9][0-9].clm2.h0.2010-02-01-00000.nc'))
    ]

    all_data = []  # List to store data from all simulations
    
    for cluster_list in cluster_lists:
        # Exclude the first element of the cluster_list
        cluster_list = cluster_list[1:]
        
        # Read each simulation and append to the list
        ds = xr.open_mfdataset(cluster_list,
                               combine='nested',
                               preprocess=lambda ds: ds[['lat', 'lon', 'time', var]],
                               parallel=True)
        all_data.append(ds)
    
    # Combine all the datasets into a single dataset along the 'ens' and 'time' dimensions
    combined_ds = xr.concat(all_data, dim=["ens", "time"], compat='override')
    
    return combined_ds

In [9]:
read_all_simulations("LNC")

MergeError: conflicting values for variable 'LNC' on objects to be combined. You can skip this check by specifying compat='override'.